# Importación de librerias necesarias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import annotations
from typing import Tuple, List

In [ ]:
df = pd.read_excel('Datos/Aldebaran_pepsi.xlsx', sheet_name='Aldebaran_pepsi')
df.to_csv('Datos/Aldebaran_pepsi.csv', index=False)
iron = pd.read_excel('Datos/Moore.xlsx', sheet_name='Hoja1')
iron.to_csv('Datos/Moore.csv', index=False)

 Devuelve los índices i que cumplen la condición discreta de mínimo local: y[i-1] > y[i] < y[i+1]
No aplica ningún filtrado ni umbral.

In [24]:
def detect_discrete_minima_candidates(y: np.ndarray) -> np.ndarray:
    y = np.asarray(y)
    if y.size < 3:
        return np.array([], dtype=int)
    cond = (y[1:-1] < y[0:-2]) & (y[1:-1] < y[2:])
    # Ajustar offset de índices porque trabajamos con y[1:-1]
    return np.nonzero(cond)[0] + 1

In [ ]:



def _fit_poly_and_refine_min(x_win, y_win, degree, x_center):

    coeffs = np.polyfit(x_win, y_win, deg=degree)

    #print("Coeficientes:", coeffs)

    p = np.poly1d(coeffs)
    dp = p.deriv(1)
    d2p = p.deriv(2)
    d3p = p.deriv(3)

    roots = np.roots(dp)
    roots = roots[np.isclose(roots.imag, 0, atol=1e-12)].real

    if len(roots) == 0:
        return np.nan, np.nan, False, False, np.nan, np.nan

    x_min, x_max = x_win.min(), x_win.max()
    roots = roots[(roots >= x_min) & (roots <= x_max)]
    if len(roots) == 0:
        return np.nan, np.nan, False, False, np.nan, np.nan

    # Elegir la raíz más cercana al candidato discreto
    x_star = roots[np.argmin(np.abs(roots - x_center))]
    y_star = p(x_star)
    is_min = d2p(x_star) > 0

    # Derivadas en el punto refinado
    second_der = float(d2p(x_star))
    third_der = float(d3p(x_star))

    return float(x_star), float(y_star), True, bool(is_min), second_der, third_der


def find_local_minima_polyfit(
    x: np.ndarray,
    y: np.ndarray,
    window_pts: int = 20,
    degree: int = 4,
    min_points: int = None,
) -> pd.DataFrame:
    """
    Encuentra y refina mínimos locales usando un ajuste polinómico local.
    Incluye segunda y tercera derivada en el mínimo refinado.
    """

    if window_pts < degree + 1:
        raise ValueError("window_pts debe ser al menos degree+1.")
    if min_points is None:
        min_points = degree + 1
    if min_points < degree + 1:
        raise ValueError("min_points debe ser >= degree+1.")

    n = len(x)
    half = window_pts // 2

    # 1) Detectar candidatos discretos sin alterar datos
    candidates = detect_discrete_minima_candidates(y)

    rows = []
    for i in candidates:
        start = max(0, i - half)
        end = min(n, start + window_pts)
        start = max(0, end - window_pts)

        x_win = x[start:end]
        y_win = y[start:end]

        if x_win.size < min_points:
            rows.append({
                "idx_candidate": int(i),
                "x_refined": np.nan,
                "y_refined": np.nan,
                "degree": degree,
                "window_start": int(start),
                "window_end": int(end),
                "num_points": int(x_win.size),
                "root_in_window": False,
                "valid_minimum": False,
                "second_derivative": np.nan,
                "third_derivative": np.nan,
            })
            continue

        try:
            x_star, y_star, root_in_window, is_min, d2, d3 = _fit_poly_and_refine_min(
                x_win, y_win, degree, x_center=x[i]
            )
        except np.linalg.LinAlgError:
            x_star, y_star, root_in_window, is_min, d2, d3 = np.nan, np.nan, False, False, np.nan, np.nan

        rows.append({
            "idx_candidate": int(i),
            "x_refined": x_star,
            "y_refined": y_star,
            "degree": degree,
            "window_start": int(start),
            "window_end": int(end),
            "num_points": int(x_win.size),
            "root_in_window": bool(root_in_window),
            "valid_minimum": bool(is_min),
            "second_derivative": d2,
            "third_derivative": d3,
        })

    return pd.DataFrame(rows)


In [22]:
df = pd.read_csv('Datos/Aldebaran_pepsi.csv')

x = df.iloc[:,0]  # primera columna
y = df.iloc[:,1]  # segunda columna


df_min = find_local_minima_polyfit(x, y, window_pts=15, degree=2)

print(df_min.sample(10))



       idx_candidate    x_refined  y_refined  degree  window_start  \
2987           65745  4516.325914   0.361122       2         65738   
7256          172732  5583.538685   0.895698       2        172725   
15254         336940  7783.803309   0.877968       2        336933   
3973           90142  4744.638185   0.339642       2         90135   
8352          196488  5865.456007   0.903411       2        196481   
2925           64324  4502.754502   0.904478       2         64317   
7513          178427  5650.677849   0.641709       2        178420   
3294           73463  4587.851157   0.669726       2         73456   
248             2176  3968.222342   0.178354       2          2169   
4368           99881  4833.830896   0.677778       2         99874   

       window_end  num_points  root_in_window  valid_minimum  \
2987        65753          15            True           True   
7256       172740          15            True           True   
15254      336948          15        

In [ ]:
## Hacer un gráfico del espectro entre dos valores específicos de longitud de onda

x1 = 3950
x2 = 3951

mask = (x >= x1) & (x <= x2)
x_range = x[mask]
y_range = y[mask]

# Graficar
plt.figure(figsize=(8, 5))
plt.plot(x_range, y_range, label="Espectro", color="blue")

plt.xlabel("Longitud de onda (Å)")
plt.ylabel("Intensidad")
plt.title("Espectro entre 3950 y 3951 Å")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Tolerancia
tol = 0.09

# Crear lista para almacenar matches
matches = []

for i, theo in enumerate(iron.iloc[:,0]):  # recorremos las líneas teóricas
    # Filtrar mínimos observados que estén cerca de la línea teórica
    nearby = df_min[(df_min["x_refined"].notna()) & 
                    (abs(df_min["x_refined"] - theo) <= tol)]
    
    for _, obs in nearby.iterrows():
        matches.append({
            "lambda_theo": theo,                 # línea teórica (Fe I/II)
            "lambda_obs": obs["x_refined"],      # longitud observada en el espectro
            "y_obs": obs["y_refined"],           # intensidad observada
            "delta": obs["x_refined"] - theo     # diferencia entre observado y teórico
        })

# Pasar a dataframe
df_matches = pd.DataFrame(matches)
df_matches = df_matches.drop_duplicates(subset=["lambda_theo"])

#print(len(df_matches))
#print(df_matches.nunique())
#print(df_matches.sample(10))


In [ ]:
c = 299792.458

df_matches["velocity_kms"] = c * (df_matches["delta"]) / df_matches["lambda_theo"]

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_matches['y_obs'], df_matches['velocity_kms'], color='blue', alpha=0.7, edgecolor='k')

# Etiquetas y título
plt.xlabel("Intensidad observada")
plt.ylabel("Velocidad Doppler (km/s)")
plt.title("Velocidad Doppler vs Intensidad observada")

# Grid y despliegue
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()